In [3]:
import pandas as pd
import numpy as np
import glob
import os

# setting the path for joining multiple files
files = os.path.join("data/all_data_v2/", "data_*.csv")

# list of merged files returned
files = glob.glob(files)

# joining files with concat and read_csv
df = pd.concat(map(pd.read_csv, files), keys = np.arange(1, 214)).reset_index()
df.drop(columns = ["level_1"], inplace = True)
df.rename(columns = {"level_0": "location_id", "Unnamed: 0" : "datetime"}, inplace = True)

ValueError: No objects to concatenate

In [4]:
engine = pd.read_csv("data/engine_metadata.csv")

engine["ID"] = engine["FILE_ID"].str.extract(r"(\d+)")

engine.head()

FileNotFoundError: [Errno 2] No such file or directory: 'data/engine_metadata.csv'

In [31]:
engine['ID'] = pd.to_numeric(engine['ID'])
df = pd.merge(df, engine, left_on = "location_id", right_on = "ID", how = "left")
df.drop(columns = ["FILE_ID", "ID"], inplace = True)

In [39]:
df.to_csv("data/all_data.csv", index = False)

# df.to_csv("data/all_data.csv.gz", index = False, compression='gzip')

In [36]:
pd.read_csv("data/all_data.csv")

,location_id,datetime,CMP_SPEED,POWER,FUEL_FLOW,CO2,CUSTOMER_NAME,PLANT_NAME,ENGINE_ID
0,1,2021-01-01 00:00:00,7864.836031,9276.641203,1.420379,3.880132,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1
1,1,2021-01-01 01:00:00,7872.746093,9256.520108,1.416093,3.868423,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1
2,1,2021-01-01 02:00:00,7860.669743,9251.712294,1.414444,3.863916,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1
3,1,2021-01-01 03:00:00,7866.736116,9288.037729,1.418746,3.875670,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1
4,1,2021-01-01 04:00:00,7864.576346,9272.706150,1.414983,3.865390,SPIFFY,SPIRITUAL-POLECAT,ENGINE_1
...,...,...,...,...,...,...,...,...,...
1866088,213,2021-12-31 20:00:00,7823.869934,10369.894938,1.634105,3.928073,MERCIFUL,PREMIUM-MOTH,ENGINE_4
1866089,213,2021-12-31 21:00:00,7817.320593,10353.512247,1.632244,3.923601,MERCIFUL,PREMIUM-MOTH,ENGINE_4
1866090,213,2021-12-31 22:00:00,7818.344861,10352.367427,1.629405,3.916775,MERCIFUL,PREMIUM-MOTH,ENGINE_4
1866091,213,2021-12-31 23:00:00,7814.145063,10331.304330,1.628703,3.915089,MERCIFUL,PREMIUM-MOTH,ENGINE_4


## add continent data

In [2]:
import numpy as np
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import pycountry_convert as pc

def get_continent(lat, lon):
    locator = Nominatim(user_agent="test@gmail.com", timeout=10)
    rgeocode = RateLimiter(locator.reverse, min_delay_seconds=1)

    location = locator.reverse(f"{lat}, {lon}")
    if location is None:
        return "Antarctica", "Antarctica", "Antarctica"
    addr = location.raw["address"]
    country_name = addr["country"]
    country_code = addr['country_code'].upper()
    continent = pc.country_alpha2_to_continent_code(country_code)
    return country_name, country_code, continent

site_df = pd.read_csv("../data/site_metadata.csv")

# create two new columns country_code and continent with get_continent
site_df["COUNTRY"], site_df["COUNTRY_CODE"], site_df["CONTINENT"] = zip(*site_df.apply(lambda x: get_continent(x["LATITUDE"], x["LONGITUDE"]), axis=1))
site_df.to_csv("../data/site_metadata.csv", index=False)

# convert continent code to full continent name in site_df.continent
def convert_continent(continent_code):
    continent_dict = {
        "NA": "North America",
        "SA": "South America",
        "AS": "Asia",
        "AF": "Africa",
        "OC": "Oceania",
        "EU": "Europe",
        "Antarctica" : "Antarctica"
    }
    return continent_dict[continent_code]

site_df["CONTINENT"] = site_df["CONTINENT"].apply(convert_continent)

NameError: name 'pd' is not defined